In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from xml.etree import ElementTree as ET
from statsmodels.stats.multitest import multipletests

In [41]:
df=pd.read_csv("csv3new_05_11_without_correct_pvalue_xiugai.csv")

In [42]:
df

,Unnamed: 0.1,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Position of SNP,P,p-value
0,0,0,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851794.0,0.000000,0.034030
1,1,1,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851805.0,0.000000,0.034030
2,2,2,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851827.0,0.000000,0.034030
3,3,3,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851833.0,0.000000,0.034030
4,4,4,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851838.0,0.304392,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
500079,500079,500079,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879736.0,0.000000,0.043326
500080,500080,500080,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879740.0,0.000000,0.043326
500081,500081,500081,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879795.0,0.000000,0.043326
500082,500082,500082,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879837.0,-0.089103,1.000000


In [43]:
import pandas as pd

df_floats = df[df['Position of gene'].apply(lambda x: isinstance(x, float))]


In [44]:
df_floats

,Unnamed: 0.1,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Position of SNP,P,p-value
30594,30594,30594,BEB,0,chr2:56405225-56644106,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33678,33678,33678,BEB,0,chr4:115247002-115485969,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86316,86316,86316,CHB,0,chr4:115247002-115485969,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94826,94826,94826,CHB,0,chr22:16175432-16345129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133155,133155,133155,CLM,0,chr12:61013177-61213757,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166055,166055,166055,GBR,0,chr2:56405225-56644106,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253824,253824,253824,JPT,0,chr22:16175432-16345129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291667,291667,291667,MXL,0,chr5:11999758-12263260,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296402,296402,296402,MXL,0,chr12:73338587-73586502,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326573,326573,326573,PJL,0,chr17:52575391-52861780,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_without_floats = df.drop(df_floats.index)
df_without_floats

,Unnamed: 0.1,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Position of SNP,P,p-value
0,0,0,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851794.0,0.000000,0.034030
1,1,1,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851805.0,0.000000,0.034030
2,2,2,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851827.0,0.000000,0.034030
3,3,3,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851833.0,0.000000,0.034030
4,4,4,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851838.0,0.304392,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
500079,500079,500079,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879736.0,0.000000,0.043326
500080,500080,500080,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879740.0,0.000000,0.043326
500081,500081,500081,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879795.0,0.000000,0.043326
500082,500082,500082,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879837.0,-0.089103,1.000000


In [46]:
pvalue_list=df_without_floats["p-value"].tolist()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
dfnew = df_without_floats.assign(corr_pvals=corr_pvals)

In [47]:
dfnew

,Unnamed: 0.1,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Position of SNP,P,p-value,corr_pvals
0,0,0,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851794.0,0.000000,0.034030,0.161256
1,1,1,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851805.0,0.000000,0.034030,0.161256
2,2,2,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851827.0,0.000000,0.034030,0.161256
3,3,3,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851833.0,0.000000,0.034030,0.161256
4,4,4,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,89851838.0,0.304392,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500079,500079,500079,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879736.0,0.000000,0.043326,0.161256
500080,500080,500080,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879740.0,0.000000,0.043326,0.161256
500081,500081,500081,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879795.0,0.000000,0.043326,0.161256
500082,500082,500082,YRI,1,chr6:32589647-32882258,ENSG00000234515,PPP1R2P1,chr6:32879171-32879848,R,32879837.0,-0.089103,1.000000,1.000000


In [48]:
dfnew.to_csv("csv3withcorr_withoutfloat.csv")

In [49]:
df_floats.to_csv("csv3_df_floats.csv")
